In [ ]:
# install osmnx through pip, conda version is too old
# osmnx version should be 1.2.2 and shapely 2.0.0

# example plot using distance from a point (edges are not contained)
import osmnx as ox
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))

# ETH 47.377126, 8.5465143
# bbox = ox.utils_geo.bbox_from_point((47.377126, 8.5465143), dist=500)

# EPFL 46.5186199,6.567573
bbox = ox.utils_geo.bbox_from_point((46.5186199,6.567573), dist=500)

poly = ox.utils_geo.bbox_to_poly(*bbox)

buildings = ox.geometries_from_polygon(poly, tags={'building': True})
roads = ox.graph_from_polygon(poly)
nodes, edges = ox.graph_to_gdfs(roads)
# G = ox.graph_from_point((47.6710676, 9.0544013), dist=1000, network_type='all')

buildings.plot(ax=ax, facecolor='black', alpha=1)
edges.plot(ax=ax, linewidth=1, edgecolor='red')

In [ ]:
# downloading from swisstopo using a GPS bounding box

import swisstopo_helpers as sw
from pathlib import Path

# from pyproj import Transformer

# # from gps to lv95
# trans = Transformer.from_crs(4326, 2056)

def transform_osm_to_sw(north, south, east, west):
  # [LONGITUDE_WEST, LAT_SOUTH, LONG_EAST, LAT_NORTH]
  return [west, south, east, north]

class Arguments(object):
  date_range = None
  bbox = None
  resolution = 0.1
  save_dir = Path("./swisstopo").resolve().absolute()

args = Arguments()

#use bounding box from above
args.bbox = transform_osm_to_sw(*bbox)
print(args.bbox)
url = sw.get_url(args)
print(url)
sw.download_tifs(url, args)


In [ ]:
# ploting the segmentation map from a swissimage tiff file with api
import re
import json
from pathlib import Path
import numpy as np
import cv2
import io


for entry in args.save_dir.iterdir():
  if(entry.name.find("swissimage-dop10") == 0 and entry.name.endswith(".tif")):
    # todo, maybe skip if there are multiple time variants of this image?
    print(entry.name)
    image_id = re.search("swissimage-dop10_([0-9]+)_[0-9]+-[0-9]+", entry.name)[0]
    json_file = image_id + ".json"
    print(json_file)
    fileObject = open(args.save_dir.joinpath(json_file), "r")
    jsonContent = fileObject.read()
    coordList = json.loads(jsonContent)
    print(coordList)


    fig, ax = plt.subplots(figsize=(5, 10))

    # using the bounding box of the api TODO
    
    # might want to add some margin to also draw streets that go out of the image
    bbox = [coordList[1][1], coordList[0][1], coordList[0][0], coordList[3][0]]
    #       47.3684871,      47.3776072,      8.5375049,       8.5509303          sample values for zurich center

    # add relative margin to bounding box
    margin = 0.2
    deltaLatMargin = (bbox[1]-bbox[0]) * margin
    deltaLongMargin = (bbox[2]-bbox[3]) * margin
    bboxM = [bbox[0] - deltaLatMargin, bbox[1] + deltaLatMargin, bbox[2] + deltaLongMargin, bbox[3] - deltaLongMargin]

    poly = ox.utils_geo.bbox_to_poly(*bboxM)
    print(poly)

    # load osm infos TODO tags
    buildings = ox.geometries_from_polygon(poly, tags={'building': True})
    roadnetwork = ox.graph_from_polygon(poly)
    nodes, edges = ox.graph_to_gdfs(roadnetwork)

    # plot osm data TODO road width
    buildings.plot(ax=ax, color='black', facecolor='black', edgecolor='black', alpha=1)
    edges.plot(ax=ax, linewidth=1, edgecolor='red')

    # corner dots
    ax.scatter([x for (x,y) in coordList], [y for (x,y) in coordList], s=40, c="green", alpha=0.5)
    ax.scatter([x for (x,y) in coordList], [y for (x,y) in coordList], s=5, c="red", alpha=1)

    # crop
    ax.set_xlim((bboxM[2], bboxM[3]))
    ax.set_ylim((bboxM[0], bboxM[1]))
    ax.set_axis_off()

    # set image size
    resolution = 3000
    inchSize = 10
    dpi = resolution // inchSize
    
    # make figure resolution x resolution and remove margins from subplot
    fig.set_size_inches(inchSize, inchSize)
    fig.tight_layout(pad=-0.08)
    buffer = io.BytesIO()
    # fig.savefig(args.save_dir.joinpath(f'{image_id}_segmentation_raw.png'),dpi=dpi)
    # save to buffer to be loaded by opencv
    fig.savefig(buffer,dpi=dpi)

    # find angle to rotate the image by
    coords = np.array(coordList)
    xDir = coords[0] - coords[3]
    xDir /= [deltaLongMargin,deltaLatMargin]
    angle = - np.angle(xDir[0] + xDir[1]*1j)
    print(angle)

    # load image in opencv
    buffer.seek(0)
    image = cv2.imdecode(np.frombuffer(buffer.read(), dtype=np.uint8), cv2.IMREAD_ANYCOLOR)
    #image = cv2.imread(str(args.save_dir.joinpath(f'{image_id}_segmentation_raw.png')))

    yRes = (bboxM[1]-bboxM[0]) / resolution
    xRes = (bboxM[3]-bboxM[2]) / resolution

    # imspired by https://stackoverflow.com/questions/11627362/how-to-straighten-a-rotated-rectangle-area-of-an-image-using-opencv-in-python/48553593#48553593
    shape = ( image.shape[1], image.shape[0] )
    
    # rotate image around center and crop
    xCenter = int((coords[0][0] - bboxM[2]) / xRes)
    yCenter = int(resolution - (coords[0][1] - bboxM[0]) / yRes)

    matrix = cv2.getRotationMatrix2D( center=(xCenter, yCenter), angle=-angle / np.pi * 180, scale=1 )
    image = cv2.warpAffine( src=image, M=matrix, dsize=shape )

    # since we use the same margins on both sides, the top left corner is more or less the inverted bottom right corner
    image = image[xCenter:resolution - xCenter, yCenter:resolution - yCenter]

    cv2.imwrite(str(args.save_dir.joinpath(f'{image_id}_segmentation.png')), image)

